<a href="https://colab.research.google.com/github/iannjari/scrapbook/blob/main/Clean_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import dependencies for data import

Un-comment the code cell below to obtain the data. Blocks should be seperated to test whether they're individually working.

In [78]:
'''
import requests
import tarfile
from os import path

# Downloading the dataset

fname = 'airline_2m.tar.gz'
url = 'https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/' + fname
r = requests.get(url)
open(fname , 'wb').write(r.content)

# Extracting the dataset
tar = tarfile.open(fname)
tar.extractall()
tar.close()

# Verifying the file was extracted properly
data_path = "airline_2m.csv"
path.exists(data_path)

import pandas as pd 
df = pd.read_csv(data_path, encoding = "ISO-8859-1",
                 dtype={'Div1Airport': str, 'Div1TailNum': str, 'Div2Airport': str, 'Div2TailNum': str})
df.shape

'''

'\nimport requests\nimport tarfile\nfrom os import path\n\n# Downloading the dataset\n\nfname = \'airline_2m.tar.gz\'\nurl = \'https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/\' + fname\nr = requests.get(url)\nopen(fname , \'wb\').write(r.content)\n\n# Extracting the dataset\ntar = tarfile.open(fname)\ntar.extractall()\ntar.close()\n\n# Verifying the file was extracted properly\ndata_path = "airline_2m.csv"\npath.exists(data_path)\n\nimport pandas as pd \ndf = pd.read_csv(data_path, encoding = "ISO-8859-1",\n                 dtype={\'Div1Airport\': str, \'Div1TailNum\': str, \'Div2Airport\': str, \'Div2TailNum\': str})\ndf.shape\n\n'

Getting the features (these were obtained by reviewing the data by datatype, description from the US DoT(helped remove redudant features) on excel, removing unwanted columns, re-reading the data and converting them to the dictionary below)

In [79]:
import pandas as pd

'''path = "airline_2m.csv"
df = pd.read_csv(path, encoding = "ISO-8859-1",
                 dtype={'Div1Airport': str, 'Div1TailNum': str, 'Div2Airport': str, 'Div2TailNum': str})
df.shape'''

'path = "airline_2m.csv"\ndf = pd.read_csv(path, encoding = "ISO-8859-1",\n                 dtype={\'Div1Airport\': str, \'Div1TailNum\': str, \'Div2Airport\': str, \'Div2TailNum\': str})\ndf.shape'

In [80]:
featurecols= ['Year','Quarter',
'Month','DayofMonth','DayOfWeek','DOT_ID_Reporting_Airline','Flight_Number_Reporting_Airline',
'OriginAirportID','OriginState','DestAirportID','DestState','DepTime','Cancelled']
featurecols

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'DOT_ID_Reporting_Airline',
 'Flight_Number_Reporting_Airline',
 'OriginAirportID',
 'OriginState',
 'DestAirportID',
 'DestState',
 'DepTime',
 'Cancelled']

In [81]:
#commmented since featuredata CSV is available featuredata=df[featurecols]
'''
featuredata=featuredata.dropna()
featuredata.to_csv('featuredata.csv',index=False)
'''

"\nfeaturedata=featuredata.dropna()\nfeaturedata.to_csv('featuredata.csv',index=False)\n"

In [82]:
featuredata=pd.read_csv('featuredata.csv')
featuredata=featuredata.dropna()
featuredata

,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Reporting_Airline,Flight_Number_Reporting_Airline,OriginAirportID,OriginState,DestAirportID,DestState,DepTime,Cancelled
0,1998,1,1,2,5,19386,675,13487,MN,14869,UT,1659.0,0.0
1,2009,2,5,28,4,20437,671,13342,WI,13204,FL,1202.0,0.0
2,2013,2,6,29,6,20398,3297,11921,CO,11298,TX,1644.0,0.0
3,2010,3,8,31,2,19790,1806,12892,CA,11433,MI,1305.0,0.0
4,2006,1,1,15,7,20355,465,11618,NJ,11057,NC,1911.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,2008,1,3,23,7,19393,966,12889,NV,14107,AZ,1444.0,0.0
1999996,1999,1,1,5,2,19704,529,11618,NJ,11298,TX,945.0,0.0
1999997,2003,4,11,14,5,20355,1457,10994,SC,11057,NC,1219.0,0.0
1999998,2012,2,5,15,2,19393,536,13232,IL,10693,TN,1838.0,0.0


In [83]:
target_name = "Cancelled"
target = featuredata[target_name]
# note that all features are categorical in nature
data = featuredata.drop(columns=[target_name])

In [84]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    data,target, test_size=0.33, random_state=42)


In [85]:
target.value_counts()

0.0    1962513
1.0        457
Name: Cancelled, dtype: int64

Class imbalance might be an issue with models modelled on this data

In [86]:
data.isnull().values.any()

False

Use a dummy classifier that predicts the majority class always as the base model

In [87]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(x_train, y_train)


dummy_clf.predict(x_test)
dummy_clf.score(x_test, y_test)

0.9997900525023118

High accuracy,arising from the imbalanced classes in the target.

What's the performance of a logistic regression?

In [88]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

logistic_regression = make_pipeline(
    OneHotEncoder(handle_unknown = 'ignore'), LogisticRegression()
)


from sklearn.model_selection import cross_validate
cv_results = cross_validate(
    logistic_regression, x_train, y_train, cv=10,return_estimator=True)

cv_results


{'estimator': (Pipeline(memory=None,
           steps=[('onehotencoder',
                   OneHotEncoder(categories='auto', drop=None,
                                 dtype=<class 'numpy.float64'>,
                                 handle_unknown='ignore', sparse=True)),
                  ('logisticregression',
                   LogisticRegression(C=1.0, class_weight=None, dual=False,
                                      fit_intercept=True, intercept_scaling=1,
                                      l1_ratio=None, max_iter=100,
                                      multi_class='auto', n_jobs=None,
                                      penalty='l2', random_state=None,
                                      solver='lbfgs', tol=0.0001, verbose=0,
                                      warm_start=False))],
           verbose=False), Pipeline(memory=None,
           steps=[('onehotencoder',
                   OneHotEncoder(categories='auto', drop=None,
                                 dtype

In [89]:
array=cv_results["test_score"]
array.mean()

0.9997559286161598

Cross-validation score of 99.97 percent

How will it perform on unseen data?

In [90]:
logistic_regression.fit(x_train, y_train)
predictions=logistic_regression.predict(x_test)
logistic_regression.score(x_test,y_test)

0.9997900525023118

Look at other metrics

In [91]:
from sklearn.metrics import confusion_matrix
c_matrix=confusion_matrix(y_test, predictions)
tn, fp, fn, tp = c_matrix.ravel()

precision= tp/(tp+fp)
misclassification= (fp+fn)/(tn+fn+tp+fp)
f_one=tp/(tp+0.5*(fp+fn))

print('Precision=',precision)
print('Misclassification=',misclassification)
print('F1 score=',f_one)

Precision= nan
Misclassification= 0.00020994749768826193
F1 score= 0.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in long_scalars
  """


In [92]:
tn,fp,fn,tp

(647645, 0, 136, 0)

In [93]:
y_test.value_counts()

0.0    647645
1.0       136
Name: Cancelled, dtype: int64

Model learnt to predict most common class due to class imbalance

In [94]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)

# fit predictor and target 

x_ros, y_ros = ros.fit_resample(x_train, y_train)

y_ros=pd.DataFrame(y_ros)

print('Original dataset shape', y_train.value_counts())
print('Resample dataset shape', y_ros.value_counts())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Original dataset shape 0.0    1314868
1.0        321
Name: Cancelled, dtype: int64
Resample dataset shape 1.0    1314868
0.0    1314868
dtype: int64


In [95]:
logistic_regression2 = make_pipeline(
    OneHotEncoder(handle_unknown = 'ignore'), LogisticRegression(max_iter=1000)
)


cv_results = cross_validate(
    logistic_regression2, x_ros, y_ros[0], cv=10,return_estimator=True)

cv_results

{'estimator': (Pipeline(memory=None,
           steps=[('onehotencoder',
                   OneHotEncoder(categories='auto', drop=None,
                                 dtype=<class 'numpy.float64'>,
                                 handle_unknown='ignore', sparse=True)),
                  ('logisticregression',
                   LogisticRegression(C=1.0, class_weight=None, dual=False,
                                      fit_intercept=True, intercept_scaling=1,
                                      l1_ratio=None, max_iter=1000,
                                      multi_class='auto', n_jobs=None,
                                      penalty='l2', random_state=None,
                                      solver='lbfgs', tol=0.0001, verbose=0,
                                      warm_start=False))],
           verbose=False), Pipeline(memory=None,
           steps=[('onehotencoder',
                   OneHotEncoder(categories='auto', drop=None,
                                 dtyp

In [96]:
array=cv_results["test_score"]
array.mean()

0.997915380124723

Cross-validation score of 99.79 percent

How will it perform on unseen data?

In [97]:
logistic_regression2.fit(x_ros,y_ros[0])
predictions=logistic_regression2.predict(x_test)
logistic_regression2.score(x_test,y_test)

0.995901392600277

In [98]:
c_matrix=confusion_matrix(y_test, predictions)
tn, fp, fn, tp = c_matrix.ravel()
precision= tp/(tp+fp)
misclassification= (fp+fn)/(tn+fn+tp+fp)
f_one=tp/(tp+0.5*(fp+fn))

print('Precision=',precision)
print('Misclassification=',misclassification)
print('F1 score=',f_one)

Precision= 0.0003966679888932963
Misclassification= 0.004098607399723055
F1 score= 0.0007527286413248024


In [99]:
tp,tn,fp,fn

(1, 645125, 2520, 135)

Try undersampling

In [100]:
# Training will be done on smaller dataset

# Create dataframe from oversampled data
df=pd.DataFrame(x_ros)
df['target']=y_ros
df

,0,1,2,3,4,5,6,7,8,9,10,11,target
0,1989,1,1,2,1,19707,52,13303,FL,11433,MI,1915,0.0
1,2019,2,5,6,1,19790,1789,12451,FL,10397,GA,2031,0.0
2,2018,3,7,29,7,20366,4150,11618,NJ,13296,NH,1958,0.0
3,2001,4,11,8,4,19393,1768,12889,NV,14869,UT,1400,0.0
4,2003,1,3,31,1,19805,1958,12892,CA,14771,CA,1900,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2629731,2015,4,12,26,6,20366,2842,11298,TX,10781,LA,2019,1.0
2629732,2010,3,8,12,4,20355,1186,11057,NC,10821,MD,1645,1.0
2629733,2013,4,12,5,4,19930,153,10299,AK,13970,AK,1755,1.0
2629734,2013,4,12,16,1,20398,3039,14783,MO,13930,IL,1713,1.0


In [101]:
# Sample data
df2=df.sample(n=300000)
df2['target'].value_counts()

0.0    150041
1.0    149959
Name: target, dtype: int64

In [137]:
x,y = df2.drop(columns=['target']),df2['target']
x_train, x_test, y_train, y_test = train_test_split(x,y,stratify=y,random_state=42)

Try stacked classifiers

In [103]:
from sklearn.ensemble import (RandomForestClassifier, StackingClassifier, GradientBoostingClassifier)
from sklearn.linear_model import LogisticRegression
from  sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [104]:


clf=StackingClassifier(estimators==[
            ("rf",RandomForestClassifier(n_estimators=10,random_state=42)),
            ("gb",GradientBoostingClassifier(n_estimators=10,random_state=42)),
            ("knn",KNeighborsClassifier(n_neighbors=5))],final_estimator=LogisticRegression())

In [108]:
pipeline = make_pipeline(
    OneHotEncoder(handle_unknown = 'ignore'), StackingClassifier(estimators=[
            ("rf",RandomForestClassifier(n_estimators=10,random_state=42)),
            ("gb",GradientBoostingClassifier(n_estimators=10,random_state=42)),
            ("knn",KNeighborsClassifier(n_neighbors=5))],final_estimator=LogisticRegression()))

pipeline.fit(x_train,y_train)
pipeline.score(x_test,y_test)

0.9999733333333334

In [109]:
preds=pipeline.predict(x_test)
c_matrix=confusion_matrix(y_test, preds)
tn, fp, fn, tp = c_matrix.ravel()
(tn, fp, fn, tp)

(37508, 2, 0, 37490)

In [110]:
precision= tp/(tp+fp)
misclassification= (fp+fn)/(tn+fn+tp+fp)
f_one=tp/(tp+0.5*(fp+fn))

print('Precision=',precision)
print('Misclassification=',misclassification)
print('F1 score=',f_one)

Precision= 0.9999466552864611
Misclassification= 2.6666666666666667e-05
F1 score= 0.999973326931797


Save this model in ONNX format

In [111]:
pip install skl2onnx

In [167]:
x_train.columns=featurecols[:-1]
x_train.to_numpy()

array([[2009, 1, 1, ..., 11292, 'CO', 544.0],
       [1989, 1, 3, ..., 10599, 'AL', 1854.0],
       [2016, 3, 8, ..., 15380, 'MI', 2015.0],
       ...,
       [2020, 1, 1, ..., 13931, 'VA', 2319.0],
       [2006, 2, 5, ..., 10397, 'GA', 930.0],
       [2010, 2, 4, ..., 14869, 'UT', 723.0]], dtype=object)

In [168]:
import skl2onnx 
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType, StringTensorType



initial_types =  [('feature_input', FloatTensorType([None, 8])),
                  ('feature_input', StringTensorType([None,1])),
                  ('feature_input', FloatTensorType([None, 1])),
                  ('feature_input', StringTensorType([None,1])),
                  ('feature_input', FloatTensorType([None, 1]))]

onx = convert_sklearn(pipeline,initial_types=initial_types)

with open("stacked_clf.onnx", "wb") as f:
    f.write(onx.SerializeToString())

TypeError: ignored